In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score
from math import log

/Users/atreish/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data set

In [2]:
x1 = np.array([ 0.        ,  3.33333333,  6.66666667, 10.        ,  0.        ,
        3.33333333,  6.66666667, 10.        ,  0.        ,  3.33333333,
        6.66666667, 10.        ,  0.        ,  3.33333333,  6.66666667,
       10.        ])
x2 = np.array([0.        , 0.        , 0.        , 0.        , 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 1.        , 1.        , 1.        ,
       1.        ])
y = np.array([ 1.22353179,  3.57574634,  5.89217837,  8.19889324,  1.96307308,
        4.57556858,  7.48364786,  9.2400334 ,  2.9048125 ,  5.74174653,
        8.19832673, 10.69424417,  4.15380028,  7.12275093,  8.8976015 ,
       11.76820779])

# 1.A

`Run a linear regression to find A, B1, and B2.`

In [3]:
dataframe = pd.DataFrame()
dataframe['x1'] = x1
dataframe['x2'] = x2
dataframe['y']  = y
X = np.array(dataframe[['x1','x2']])
X

array([[ 0.        ,  0.        ],
       [ 3.33333333,  0.        ],
       [ 6.66666667,  0.        ],
       [10.        ,  0.        ],
       [ 0.        ,  0.33333333],
       [ 3.33333333,  0.33333333],
       [ 6.66666667,  0.33333333],
       [10.        ,  0.33333333],
       [ 0.        ,  0.66666667],
       [ 3.33333333,  0.66666667],
       [ 6.66666667,  0.66666667],
       [10.        ,  0.66666667],
       [ 0.        ,  1.        ],
       [ 3.33333333,  1.        ],
       [ 6.66666667,  1.        ],
       [10.        ,  1.        ]])

In [4]:
Reg1 = LinearRegression().fit(X, y) 
coefs = Reg1.coef_
intercept = Reg1.intercept_
print('The Intercept A = {}'.format(intercept))
print('\nThe Coefficients B1, B2 = {} , {}'.format(coefs[0],coefs[1]))

The Intercept A = 1.0324877889494903

The Coefficients B1, B2 = 0.738183186821566 , 3.2574629401353596


# 1.B

`Calculate SE(B1) and SE(B2)`

In [5]:
x = np.array(X)
x3 = (x - x.mean())**2
x4 = sum(x3)
x5 = np.sqrt(x4)
x5

array([17.41327718,  9.12262146])

In [6]:
k = X.shape[1]
n = X.shape[0]
y_hat = Reg1.predict(X)
Errors = y - y_hat
SSE = sum(Errors**2)
SE = np.sqrt(SSE/(n-k-1))
SE_coefs = SE / x5
print('Standard Error of B1 = {}'.format(SE_coefs[0]))
print('Standard Error of B2 = {}'.format(SE_coefs[1]))

Standard Error of B1 = 0.013883328050068445
Standard Error of B2 = 0.02650052298514973


# 1.C

`Calculate a prediction, y_hat, at x1=5, x2=.5 along with $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$`

In [7]:
Y_hat1 = Reg1.predict(np.array([[5,0.5]]))
print('Y_hat at x1=5 and x2=0.5 is = {}'.format(Y_hat1))

Y_hat at x1=5 and x2=0.5 is = [6.35213519]


In [8]:
RSS = sum((y - Y_hat1)**2)
SE_yhat = (np.sqrt(RSS/(n-k-1)))/np.sqrt(n)
print('The Standard Error of our Y_hat calculation is = {}'.format(SE_yhat))

The Standard Error of our Y_hat calculation is = 0.8361772658727425


# 2.A

`Calculate m=100 bootstrap sample sets from the original set of 16 samples and, for each bootstrap sample set, estimate A_bs, B1_bs, and B2_bs by regression. (You'll have 100 estimates of each of A_bs, B1_bs, and B2_bs.`

In [9]:
len(x)

16

In [10]:
def bootstrap(x, y):
    i = np.random.randint(0, 16, size=16)
    return x[i], y[i]

m = 100
A_bs = []
B1_bs = []
B2_bs = []
bootstrap_x1 = []
bootstrap_y1 = []
all_bs = []

for _ in range(m):     
    bootstrap_x, bootstrap_y = bootstrap(x, y)
    Reg2 = LinearRegression().fit(bootstrap_x, bootstrap_y)
    A_bs.append(Reg2.intercept_)
    B1_bs.append(Reg2.coef_[0])
    B2_bs.append(Reg2.coef_[1])
    bootstrap_x1.append(bootstrap_x)
    bootstrap_y1.append(bootstrap_y)
    all_bs.append(Reg2)

In [11]:
print(len(A_bs)) 
print(len(B1_bs))
print(len(B2_bs))

100
100
100


# 2.B

`Compute SE(B1_bs) and SE(B2_bs) directly from the m=100 samples. Do they match SE(B1) and SE(B2) from part 1, above?`

In [12]:
k = bootstrap_x1[0].shape[1]
n = bootstrap_x1[0].shape[0]

SE_B1 = []
SE_B2 = []

for i in range(m):
    E = bootstrap_y1[i] - all_bs[i].predict(bootstrap_x1[i])
    SSE = sum(E**2)
    SE = np.sqrt( SSE/(n-k-1) )
    SE_B = SE / np.sqrt(sum((bootstrap_x1[i] - bootstrap_x1[i].mean())**2))
    SE_B1.append(SE_B[0])
    SE_B2.append(SE_B[1])

In [13]:
print('The Standard Error of the Bootstrapped B1 coefficient = {}'.format(np.mean(SE_B1)))
print('The Standard Error of the Bootstrapped B2 coefficient = {}'.format(np.mean(SE_B2)))

The Standard Error of the Bootstrapped B1 coefficient = 0.013164420315520254
The Standard Error of the Bootstrapped B2 coefficient = 0.025373253761279094


# 2.C

`For each model, A_bs, B1_bs, and B2_bs, calculate a prediction, y_hat_bs, at x1=5, x2=.5. From that set of predictions, compute SE(y_hat_bs). Does it match SE(y_hat) from part 1, above?`

In [14]:
Yhat_bs = [all_bs[i].predict(np.array([[5, 0.5]])) for i in range(m)]

SE_Yhat_bs = []

for i in range(m):
    RSS = sum((bootstrap_y1[i] - Yhat_bs[i])**2)
    SE_Yhat_bs.append((np.sqrt(RSS/(n-k-1)))/np.sqrt(n))

In [15]:
print('The Standard Error of the Bootstrapped Y_hat calculation for B1=5, B2=0.5 = {}'.format(np.mean(SE_Yhat_bs)))

The Standard Error of the Bootstrapped Y_hat calculation for B1=5, B2=0.5 = 0.8126102951312343


In [16]:
print('The Values are very close to eachother by {}'.format(SE_yhat - np.mean(SE_Yhat_bs)))

The Values are very close to eachother by 0.023566970741508264
